# TABLE OF CONTENT : 
 1. [Scrapping Wikipedia Web page](#1)<br>
 2. [Adding coordinates of neighborhoods](#2)<br>
 3. [Folium maps](#3)<br>
 4. [Cluster neighborhoods](#4)<br>

In [1]:
#Installing beautifulsoup package
#!pip install beautifulsoup4

In [1]:
#import of libraries necessary to scrap the web page 
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import folium

# 1. SCRAPPING WIKIPEDIA WEB PAGE <a id="1"></a>

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url

'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#parsing the web page wikipedia
page_response=requests.get(url,timeout=5)
page_content = BeautifulSoup(page_response.content, 'lxml')

In [4]:
#creation of the data frame containing the table from wikipedia page : 
# varibale with all the tables found in the page 
tables=page_content.find_all('table')
#focus on the table we are interested in : 
table=tables[0].tbody
#creation of our data frame :
df_table=pd.DataFrame(columns=['PostalCode','Borough','Neighborhood'])
#iteration in the table in order to extract content and add it to our dataframe 
for row in table.find_all('tr') : 
    cols=row.find_all('td')
    try : 
        df_table=df_table.append({'PostalCode':cols[0].text,
                                          'Borough': cols[1].text, 
                                          'Neighborhood': cols[2].text, 
                                         }, ignore_index=True)
    except: 
        pass

df_table

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,Regent Park / Harbourfront\n
...,...,...,...
175,M5Z\n,Not assigned\n,\n
176,M6Z\n,Not assigned\n,\n
177,M7Z\n,Not assigned\n,\n
178,M8Z\n,Etobicoke\n,Mimico NW / The Queensway West / South of Bloo...


In [5]:
#data frame cleaning by removin the '\n' using  REGEX: 
import re
for index,row in df_table.iterrows():
    for col in df_table.columns:
        row[col]= re.sub(r'\n','',row[col])
df_table

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
...,...,...,...
175,M5Z,Not assigned,
176,M6Z,Not assigned,
177,M7Z,Not assigned,
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


In [7]:
#drop rows where no borough were attributed and copy borough neighborhood where no neighborood
for index, row in df_table.iterrows():
    row['Neighborhood']=row['Neighborhood'].replace('/',',')
    if row['Borough']=='Not assigned':
        df_table.drop([index],inplace=True)
    if row['Neighborhood']== '':
        row['Neighborhood']=row['Borough']
df_table.reset_index(inplace=True,drop=True)
df_table.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
# checking if any null values are stil in the data frame : 
df_clean=df_table.isnull()
for column in df_clean.columns: 
    print(column)
    print(df_clean[column].value_counts())

PostalCode
False    103
Name: PostalCode, dtype: int64
Borough
False    103
Name: Borough, dtype: int64
Neighborhood
False    103
Name: Neighborhood, dtype: int64


In [9]:
df_table.shape

(103, 3)

In [134]:
#export data to csv
import csv
#df_table.to_csv('postal_code.csv')

# 2. Adding coordinates of neighborhoods <a id="2"></a>


In [11]:
# instal of geocoder 
#!pip install geocoder

In [14]:
import geocoder # import geocoder

In [15]:
# initialize your variable to None
lat_lng_coords = None
for index,code in enumerate(df_table['PostalCode']): 
    postal_code=code
    print(index, 'st row to complete')
# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.geocodefarm('{}, Toronto, Ontario,Canada'.format(postal_code))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    df_table.at[index,'latitude'] = lat_lng_coords[0]
    df_table.at[index,'longitude'] = lat_lng_coords[1]
    print(index, 'st row completed')
    lat_lng_coords = None

0 st row to complete
[43.7518806457716, -79.3303604125129]
0 st row completed
1 st row to complete
[43.7304191589716, -79.3128204341299]
1 st row completed
2 st row to complete
None
None
[43.6551399230715, -79.362648010213]
2 st row completed
3 st row to complete
[43.7232093811716, -79.4514083861301]
3 st row completed
4 st row to complete
None
[43.6644897460715, -79.393020629813]
4 st row completed
5 st row to complete
[43.6627693176715, -79.528312683113]
5 st row completed
6 st row to complete


KeyboardInterrupt: 

In [253]:
# extraction of the dataframe into a csv file
#df_table.to_csv('postal_code.csv')

# 3. Folium Maps <a id="3"></a>

In [18]:
#reading the previously completed csv with cordinates
df_table=pd.read_csv('postal_code.csv',index_col=0)
df_table

,PostalCode,Borough,Neighborhood,latitude,longitude
0,M3A,North York,Parkwoods,43.751881,-79.330360
1,M4A,North York,Victoria Village,43.730419,-79.312820
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.655140,-79.362648
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.723209,-79.451408
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.664490,-79.393021
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653690,-79.511124
99,M4Y,Downtown Toronto,Church and Wellesley,43.666592,-79.381302
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.648689,-79.385437
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.632881,-79.489548


In [23]:
#checking data with folium maps of neigborhoods
#1st coordinates of Toronto
g2 = geocoder.geocodefarm('Toronto, Ontario,Canada')
latlong= g2.latlng
latitude= latlong[0]
longitude = latlong[1]
print(f'latitude {latitude} & longitude {longitude}')

latitude 43.6486892707151 & longitude -79.385437011713


In [24]:
#folium map : 
toronto_map= folium.Map(location=[latitude, longitude],tiles='OpenStreetMap', zoom_start=10)
for borough,neighborhood, lat,long in zip(df_table['Borough'],df_table['Neighborhood'],df_table['latitude'],df_table['longitude']):
    label=(f'{borough}, {neighborhood}')
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(location=[lat,long],
                        popup=label,
                        radius=5, 
                        fill=True,
                        fill_color='#3388ff',
                        fill_opacity=1,
                        parse_html=False).add_to(toronto_map)
    
toronto_map

# 4. Cluster neighborhoods<a id="4"></a>

In [25]:
#import librairies : 
# library to handle JSON files
import json 
 # library to handle requests
import requests 
from sklearn.cluster import kMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

ImportError: cannot import name 'kMeans' from 'sklearn.cluster' (c:\users\pierre beylard\appdata\local\programs\python\python37\lib\site-packages\sklearn\cluster\__init__.py)

In [32]:
#GET INFORMATION ON FOURSQUARE API
#FOURSQUARE CREDENTIALS
CLIENT_ID = 'CUF5UKNECLDYIDGLLSJJF32FI1RYQJBR1WRLHIFQEVRSJJO4' 
CLIENT_SECRET = 'SW40I4VDCJKKBFFXLQOFRMBRW0AQ1DWO3ZO23QF3JUXLKWFJ' 
VERSION = '20180605' 

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#foursquare url : 
url_base='https://api.foursquare.com/v2/'

Credentails:
CLIENT_ID: CUF5UKNECLDYIDGLLSJJF32FI1RYQJBR1WRLHIFQEVRSJJO4
CLIENT_SECRET:SW40I4VDCJKKBFFXLQOFRMBRW0AQ1DWO3ZO23QF3JUXLKWFJ


In [26]:
Toronto_data=pd.DataFrame(columns=df_table.columns[1:])

In [27]:
for index,row in enumerate(df_table['Borough']):
    if 'Toronto' in row: 
        Toronto_data=Toronto_data.append(df_table.loc[index,'Borough':'longitude'])
Toronto_data.reset_index(inplace=True,drop=True)

In [28]:
Toronto_data

,Borough,Neighborhood,latitude,longitude
0,Downtown Toronto,"Regent Park , Harbourfront",43.655140,-79.362648
1,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.664490,-79.393021
2,Downtown Toronto,"Garden District, Ryerson",43.657360,-79.378181
3,Downtown Toronto,St. James Town,43.651428,-79.375572
4,East Toronto,The Beaches,43.677029,-79.295418
5,Downtown Toronto,Berczy Park,43.645309,-79.373680
6,Downtown Toronto,Central Bay Street,43.656090,-79.384933
7,Downtown Toronto,Christie,43.668781,-79.420708
8,Downtown Toronto,"Richmond , Adelaide , King",43.649700,-79.382584
9,West Toronto,"Dufferin , Dovercourt Village",43.665089,-79.438713


In [30]:
# visualisation of Toronto Boroughs : 

borough= folium.Map(location=[latitude,longitude],zoom_start=11)
for lat,long,bo,nei in zip(Toronto_data['latitude'],Toronto_data['longitude'],Toronto_data['Borough'],Toronto_data['Neighborhood']):
    label=(bo +' -\n'+ nei)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(location=[lat,long],
                        popup=label,
                        radius=5,
                        fill=True,
                        fill_color='#3388ff',
                        fill_opacity=1,
                        parse_html=False
                       ).add_to(borough)
borough

In [126]:
# connect to foursquare and get the venues for all neigborhoods
RADIUS=500
LIMIT= 100
temp=pd.DataFrame()
resultat = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood', 'latitude', 'longitude','venu_name','venue_category'])
for code, bo, nei,lat,long in zip(df_table['PostalCode'],df_table['Borough'],df_table['Neighborhood'],df_table['latitude'],df_table['longitude']):
    url=url_base+f'venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={RADIUS}&limit={LIMIT}'
    results=requests.get(url).json()    
    for i in range(len(result['response']['groups'][0]['items'])):
        temp.at[i,'PostalCode']=code
        temp.at[i,'Borough']=bo
        temp.at[i,'Neighborhood']=nei
        temp.at[i,'latitude']=lat
        temp.at[i,'longitude']=long
        temp.at[i,'venu_name']=result['response']['groups'][0]['items'][i]['venue']['name']
        temp.at[i,'venue_category']=result['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']
    resultat=resultat.append(temp,ignore_index=True)
resultat

,PostalCode,Borough,Neighborhood,latitude,longitude,venu_name,venue_category
0,M3A,North York,Parkwoods,43.751881,-79.330360,Roselle Desserts,Bakery
1,M3A,North York,Parkwoods,43.751881,-79.330360,Tandem Coffee,Coffee Shop
2,M3A,North York,Parkwoods,43.751881,-79.330360,Figs Breakfast & Lunch,Breakfast Spot
3,M3A,North York,Parkwoods,43.751881,-79.330360,Morning Glory Cafe,Breakfast Spot
4,M3A,North York,Parkwoods,43.751881,-79.330360,Cocina Economica,Mexican Restaurant
...,...,...,...,...,...,...,...
2467,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,Vistek,Electronics Store
2468,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,Savoury Grounds,Coffee Shop
2469,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,Flame Shack,Restaurant
2470,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,The Healthy Road,Health Food Store


In [171]:
resultat

,PostalCode,Borough,Neighborhood,latitude,longitude,venu_name,venue_category
0,M3A,North York,Parkwoods,43.751881,-79.330360,Roselle Desserts,Bakery
1,M3A,North York,Parkwoods,43.751881,-79.330360,Tandem Coffee,Coffee Shop
2,M3A,North York,Parkwoods,43.751881,-79.330360,Figs Breakfast & Lunch,Breakfast Spot
3,M3A,North York,Parkwoods,43.751881,-79.330360,Morning Glory Cafe,Breakfast Spot
4,M3A,North York,Parkwoods,43.751881,-79.330360,Cocina Economica,Mexican Restaurant
...,...,...,...,...,...,...,...
2467,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,Vistek,Electronics Store
2468,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,Savoury Grounds,Coffee Shop
2469,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,Flame Shack,Restaurant
2470,M8Z,Etobicoke,"Mimico NW , The Queensway West , South of Bloo...",43.624630,-79.528351,The Healthy Road,Health Food Store


In [170]:
# group by neighborhoods the most venues category 
dummies=pd.get_dummies(resultat[['venue_category']], prefix="", prefix_sep="")
dummies['Neighborhood']=resultat['Neighborhood']
#getting last column at the bigining (neigborhoods):
ordered_columns=[dummies.columns[-1]]+ list(dummies.columns[:-1])
dummies=dummies[ordered_columns]
dummies
dummies_grouped=dummies.groupby('Neighborhood').sum().reset_index()
dummies_grouped

,Neighborhood,Bakery,Breakfast Spot,Coffee Shop,Distribution Center,Electronics Store,Event Space,Food Truck,Gym / Fitness Center,Health Food Store,Italian Restaurant,Mexican Restaurant,Pub,Restaurant,Spa,Thai Restaurant,Theater,Wine Shop,Yoga Studio
0,Agincourt,1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
1,"Alderwood , Long Branch",1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
2,"Bathurst Manor , Wilson Heights , Downsview North",1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
3,Bayview Village,1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
4,"Bedford Park , Lawrence Manor East",1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Willowdale , Newtonbrook",1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
94,Woburn,1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
95,Woodbine Heights,1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1
96,"York Mills , Silver Hills",1,2,5,1,1,1,1,1,1,1,1,1,2,1,1,1,1,1


In [197]:
# create a single dataframe with each neighborhoods and its venues with coordinates 
# group by neighborhoods the most venues category 
# apply clustering algorithm on neighborhoods based on most common categories 

(105, 3)

# Code pour boucler sur le dataframe et fusionner les neighborhoods ayant les mêmes code postals 

In [242]:
df_test=df_table[['PostalCode','Borough','Neighborhood']].copy()
#ajout de deux lignes supplémentaires avec même code postal pour l'exercice : 
df_test=df_test.append(pd.DataFrame({'PostalCode':['M3A','M3A'],'Borough':['North York','North York'],'Neighborhood':['Pierre','Sarah']}))
df_test.reset_index(inplace=True,drop=True)

In [243]:
#on trie les valeurs pour que les codes postaux identiques soient à la suite dans le data frame : 
df_test=df_test.sort_values(by=['PostalCode'])
df_test.reset_index(inplace=True,drop=True)
# Boucle pour trouver les codes postals identiques et fusionner les neigborhoods: 
for index, value in enumerate(df_test['PostalCode']):
    try : 
        i=1
        if index!= len(df_test['PostalCode']) :
            while value == df_test.loc[index+i, 'PostalCode']:  
                df_test.at[index,'Neighborhood']+= ', '+ (df_test.at[index+i,'Neighborhood'])
                df_test.drop([index+i],inplace=True)
                i+=1     
    except : 
        print('fin du dataframe')
                       

fin du dataframe
fin du dataframe


In [244]:
df_test[df_test['PostalCode']=='M3A']

,PostalCode,Borough,Neighborhood
25,M3A,North York,"Pierre, Parkwoods, Sarah"


In [210]:
df_test

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern , Rouge"
12,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
18,M1E,Scarborough,"Guildwood , Morningside , West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
...,...,...,...
64,M9N,York,Weston
70,M9P,Etobicoke,Westmount
77,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
89,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [92]:
# coordinates from csv geospatial
df_table.columns
df_2=pd.read_csv('Geospatial_Coordinates.csv')
df_2.rename(columns={'Postal Code': 'PostalCode'},inplace=True)
df_3=pd.merge(df_table[['PostalCode', 'Borough', 'Neighborhood']],df_2, on='PostalCode',how='inner')

In [94]:
#folium map : 
toronto_map= folium.Map(location=[latitude, longitude],tiles='OpenStreetMap', zoom_start=12)
for borough,neighborhood, lat,long in zip(df_3['Borough'],df_3['Neighborhood'],df_3['Latitude'],df_3['Longitude']):
    label=(f'{borough}, {neighborhood}')
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(location=[lat,long],
                        popup=label,
                        radius=5, 
                        fill=True,
                        fill_color='#3388ff',
                        fill_opacity=1).add_to(toronto_map)
    
toronto_map